In [3]:
import pandas as pd
import numpy as np
import sys
import os
import re

In [16]:

# Define the log file path
log_file = r"C:\Users\janbe\Documents\GitHub Repos\pulse\output_to_keep\sys_msgs_test\20250616_135330_gemma_aki_part1\log_20250616_135330.log"

# Prepare regex patterns for each field
block_start = re.compile(r"Decoded output:", re.MULTILINE)
decoded_output = re.compile(r"```json\s+({.*?})\s+```", re.DOTALL)
tokenization = re.compile(
    r"Tokenization time: ([\d.]+)s \| Inference time: ([\d.]+)s \| Input Tokens: (\d+) \| Output Tokens: (\d+)"
)
prob_label = re.compile(r"Predicted probability: ([\d.]+) \| True label: (\d+)")
diagnosis = re.compile(r"Diagnosis for: (.+)")
explanation = re.compile(r"Generated explanation: (.+)")
system_message = re.compile(r"System message: (.+)")
system_message_idx = re.compile(r"System message index: (\d+)")

In [17]:
# Read the log file
with open(log_file, "r", encoding="utf-8") as f:
    log = f.read()

# Only process after the start line
start_line = "Starting test evaluation..."
start_idx = log.find(start_line)
if start_idx != -1:
    log = log[start_idx + len(start_line) :]

In [20]:
# Split into blocks
blocks = re.split(block_start, log)
blocks = blocks[1:]  # Skip the first empty block
rows = []

In [21]:
for block in blocks:
    if not block.strip():
        continue

    # Extract fields
    decoded = decoded_output.search(block)
    diagnosis_json = None
    if decoded:
        import json

        try:
            diagnosis_json = json.loads(decoded.group(1))
        except Exception:
            diagnosis_json = {}

    token_match = tokenization.search(block)
    prob_label_match = prob_label.search(block)
    diagnosis_match = diagnosis.search(block)
    explanation_match = explanation.search(block)
    system_msg_match = system_message.search(block)
    system_msg_idx_match = system_message_idx.search(block)

    row = {
        "Input Prompt": "",  # Not present in your sample, fill as needed
        "Target Label": prob_label_match.group(2) if prob_label_match else "",
        "Predicted Probability": prob_label_match.group(1) if prob_label_match else "",
        "Predicted Diagnosis": (
            diagnosis_json.get("diagnosis", "") if diagnosis_json else ""
        ),
        "Predicted Explanation": (
            diagnosis_json.get("explanation", "") if diagnosis_json else ""
        ),
        "Tokenization Time": token_match.group(1) if token_match else "",
        "Inference Time": token_match.group(2) if token_match else "",
        "Input Tokens": token_match.group(3) if token_match else "",
        "Output Tokens": token_match.group(4) if token_match else "",
        "System Message": system_msg_match.group(1) if system_msg_match else "",
        "System Message Index": (
            system_msg_idx_match.group(1) if system_msg_idx_match else ""
        ),
    }
    rows.append(row)

# Create DataFrame
df = pd.DataFrame(
    rows,
    columns=[
        "Input Prompt",
        "Target Label",
        "Predicted Probability",
        "Predicted Diagnosis",
        "Predicted Explanation",
        "Tokenization Time",
        "Inference Time",
        "Input Tokens",
        "Output Tokens",
        "System Message",
        "System Message Index",
    ],
)

In [23]:
df["System Message Index"] = np.tile(np.arange(5), len(df))[:len(df)]
df

,Input Prompt,Target Label,Predicted Probability,Predicted Diagnosis,Predicted Explanation,Tokenization Time,Inference Time,Input Tokens,Output Tokens,System Message,System Message Index
0,,0,0.85,diagnosis,The patient exhibits several concerning signs ...,0.0069,22.4224,3786,300,,0
1,,0,0.75,aki,Acute kidney injury is possible. Creatinine is...,0.0055,21.5791,3957,300,,1
2,,0,0.78,aki,Likely acute kidney injury. Creatinine is stab...,0.0046,21.5295,4052,300,,2
3,,0,0.78,aki,Likely acute kidney injury. Creatinine is stab...,0.0044,21.5596,4084,300,,3
4,,0,0.75,aki,Likely acute kidney injury. Creatinine is stab...,0.0047,21.6247,4158,300,,4
...,...,...,...,...,...,...,...,...,...,...,...
3325,,0,0.85,diagnosis,The patient exhibits several concerning trends...,0.0044,21.4758,3790,300,,0
3326,,0,0.75,aki,Acute kidney injury is suspected. Creatinine d...,0.0044,21.5801,3961,300,,1
3327,,0,0.78,aki,Likely acute kidney injury. Creatinine decreas...,0.0051,21.4746,4056,300,,2
3328,,0,0.78,aki,Acute kidney injury is likely. Creatinine decr...,0.0045,21.4068,4088,300,,3


In [24]:
df.to_csv(
    r"C:\Users\janbe\Documents\GitHub Repos\pulse\output_to_keep\sys_msgs_test\20250616_135330_gemma_aki_part1\Gemma3_aki_20250616_135330_metadata.csv",
    index=False,
)